# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]

In [3]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
**需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。**
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
X, Y

(tensor([[ 0.4717,  0.7608, -0.2460, -0.5224,  1.8961, -0.3454, -0.0133,  1.5230,
          -0.3957, -0.1714, -1.8560,  0.7084, -0.0031,  0.1693,  0.1832,  0.6183,
          -0.5887,  1.9986,  0.7505, -0.1436],
         [-0.1473,  0.1644,  0.2538,  0.7431,  0.4841,  0.0939, -0.5621,  0.0082,
          -0.1713,  0.2585,  0.6824,  0.3604,  0.8227,  0.1294, -0.4082, -0.3461,
           0.6684, -0.6188, -0.5184,  0.4627]]),
 tensor([[ 0.0556, -0.0790,  0.0728, -0.0750,  0.1486,  0.1396, -0.2337,  0.1133,
           0.1514,  0.0022],
         [-0.0183, -0.0565,  0.0204,  0.0016, -0.0048,  0.2369,  0.0323,  0.1494,
           0.0461, -0.1177]], grad_fn=<AddmmBackward0>))

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]

In [6]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1166,  0.1421,  0.0079,  ..., -0.2129, -0.0159, -0.2110],
                      [-0.1750, -0.0147, -0.1130,  ...,  0.0023, -0.0808, -0.1055],
                      [-0.1670, -0.1670,  0.1465,  ...,  0.1507,  0.0674,  0.0513],
                      ...,
                      [ 0.1201, -0.0734, -0.1010,  ..., -0.0786,  0.1808,  0.2135],
                      [-0.1636,  0.0960, -0.0222,  ...,  0.1768,  0.1840,  0.1018],
                      [ 0.0564,  0.1881,  0.0744,  ..., -0.1790,  0.0665, -0.1621]])),
             ('hidden.bias',
              tensor([ 7.3997e-02, -7.7489e-02,  2.0512e-01,  1.0465e-01,  1.0354e-01,
                      -9.7469e-02,  1.9908e-01,  6.9771e-02, -3.3769e-02,  9.9017e-02,
                      -1.7160e-01,  1.7985e-01, -2.1898e-01,  5.4486e-02, -1.1885e-01,
                      -2.1886e-01, -2.1454e-01, -1.0504e-02,  1.7328e-01,  2.3252e-02,
                       1.1865e-01,  1.1366e-01, -2.1297e-0

In [7]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)

In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。

注：eval评估模式下dropout会被关掉

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？

### 练习一

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

**解答：**

&emsp;&emsp;1.便于模型恢复和微调：存储模型参数可以在训练中断后继续训练，或者在新的数据集上进行微调，而无需从头开始训练。

&emsp;&emsp;2.节省存储空间：相比于保存完整的模型结构，保存模型参数通常占用更少的存储空间，这在处理大型模型或存储空间受限的情况下尤为重要。

&emsp;&emsp;3.便于共享和复现：存储模型参数可以方便地共享和复现已经训练好的模型，其他人可以直接加载这些参数并使用它们进行预测或进一步训练。

&emsp;&emsp;4.便于调试和分析：通过检查和分析模型参数，可以更容易地诊断模型中存在的问题，并对模型进行调整和优化。

&emsp;&emsp;下面将以一个简单的例子具体说明。

&emsp;&emsp;1. 便于模型恢复和微调：假设我们有一个简单的神经网络模型，我们可以使用 PyTorch 保存和加载模型参数：

In [10]:
import torch
import torch.nn as nn

# 定义模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 保存模型参数
torch.save(model.state_dict(), 'model_parameters.pth')

# 加载模型参数进行微调
model.load_state_dict(torch.load('model_parameters.pth'))
# 继续训练或进行微调...

<All keys matched successfully>

&emsp;&emsp;2. 节省存储空间：在上述示例中，通过使用`torch.save(model.state_dict(), 'model_parameters.pth')`只保存模型参数，而不是整个模型，可以节省存储空间。

&emsp;&emsp;3. 便于共享和复现：保存模型参数后，可以将 `model_parameters.pth` 文件与他人共享。他们可以使用以下代码加载参数并复现模型：

In [11]:
model = SimpleModel()
model.load_state_dict(torch.load('model_parameters.pth'))
# 使用模型进行预测或进一步训练...

<All keys matched successfully>

&emsp;&emsp;4. 便于调试和分析：通过检查模型参数，可以对模型进行调试和分析。例如，可以打印出模型权重和偏置：

In [12]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param.data)

fc.weight
tensor([[-0.2898,  0.2839,  0.0169,  0.0477, -0.0595, -0.2568, -0.0884, -0.2903,
         -0.1100, -0.1530],
        [ 0.2284,  0.0527, -0.2718, -0.0059, -0.1315,  0.3001,  0.0876, -0.2056,
         -0.1616, -0.0564]])
fc.bias
tensor([-0.1099,  0.0685])


### 练习二

2. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

**解答：**

&emsp;&emsp;使用保存模型某层参数的办法，保存网络的前两层，然后再加载到新的网络中使用。

In [13]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    """定义 MLP 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 定义隐藏层，输入尺寸为20，输出尺寸为256。
        self.output = nn.Linear(256, 10) # 定义输出层，输入尺寸为256，输出尺寸为10

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

class MLP_new(nn.Module):
    """定义 MLP_new 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 定义隐藏层，输入尺寸为 20，输出尺寸为 256。
        self.output = nn.Linear(256, 10)  # 定义输出层，输入尺寸为 256，输出尺寸为 10。

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

net = MLP() # 创建 MLP 的实例
# 将隐藏层的参数保存到文件中。
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')

clone = MLP_new() # 创建另一个 MLP 的实例。
# 加载已保存的参数到克隆实例的隐藏层中。
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))

# 比较两个 MLP 示例的隐藏层权重是否相等，并输出结果
print(clone.hidden.weight == net.hidden.weight)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


### 练习三

3. 如何同时保存网络架构和参数？需要对架构加上什么限制？

&emsp;&emsp;在PyTorch中，可以使用`torch.save()`函数同时保存网络架构和参数。为了保存网络架构，需要将模型的结构定义在一个Python类中，并将该实例化为模型对象。此外，必须确保该类的构造函数不包含任何随机性质的操作，例如`dropout`层的随机丢弃率应该是固定的。

In [14]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    """定义 MLP 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 定义隐藏层，输入尺寸为 20，输出尺寸为 256。
        self.output = nn.Linear(256, 10) # 定义输出层，输入磁村为 256，输出尺寸为 10。

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

net = MLP()

# 存储模型
torch.save(net, 'model.pt')

# 导入模型
model = torch.load('model.pt')
model

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

[讲解了保存网络结构和参数；和只保存网络参数；这两种方法。](https://blog.csdn.net/huihui__huihui/article/details/107216659)